# EMEA - Run Pre Launch Model - Dev

In [0]:
sf_creds = 'hbo-max-content-datascience-snowflake-dev'
database = 'max_dev'
input_bucket = "hbo-ingest-datascience-content"
output_bucket = "hbo-outbound-datascience-content-dev"


In [0]:
# Import Packages
import sys, os, re 
import io
import pandas as pd
import numpy as np
import itertools as it
import logging
import boto3
import json
from datetime import datetime, timedelta


import lib.util_snowflake as sfk
from snowflake.connector.errors import ProgrammingError
from snowflake.connector.pandas_tools import write_pandas

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)
logger.info(f'Starting Notebook')

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [0]:
%load_ext autoreload
%autoreload 2
schema = 'content_datascience'
current_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
kpi = 'viewing_subs'
geo_value = 'EMEA'
schema = 'delphi'

In [0]:
## Run Credentials to connect to Snowflake
logger.info(f'TEST: {sf_creds}')
## Snowflake connection 
conn = sfk.SnowflakeConnector(sfk.SSMPSCredentials(sf_creds))
ctx= conn.connect(database, schema)
cur = ctx.cursor()

## 1.0 Query Data

In [0]:
# train_test_scope = 'lib/dev_train_test_scope.py'
# %run $train_test_scope

In [0]:
## 2.1 Read Metadata New
query_metadata = f"""select * from {database}.content_datascience.viewingsubs_metadata_train where geo_value='{geo_value}'"""
logger.info(f'TEST: {query_metadata}')
metadata_feature = sfk.execute_query(query = query_metadata, ctx=ctx)
metadata_feature['first_release_date'] = pd.to_datetime(metadata_feature['first_release_date']).dt.strftime('%Y-%m-%d')

## 2.2 Read Future Schedule
query_schedule = f"""select * from {database}.content_datascience.viewingsubs_metadata_pred where geo_value='{geo_value}' and imdb_series_id is not null"""
logger.info(f'TEST: {query_schedule}')
df_pred = sfk.execute_query(query = query_schedule, ctx = ctx)
# print('Loading SFK table file {}'.format(file_ref))
df_pred['first_release_date'] = df_pred['first_release_date'].combine_first(df_pred['first_international_premiere'])
df_pred['first_release_date'] = pd.to_datetime(df_pred['first_release_date']).dt.strftime('%Y-%m-%d')

In [0]:
# Temp imdbd Pull

query_schedule = f"""select * from {database}.content_datascience.viewingsubs_metadata_pred where geo_value='NORTH AMERICA' and prequel_count >0"""
logger.info(f'TEST: {query_schedule}')
df_imdb = sfk.execute_query(query = query_schedule, ctx = ctx)
# print('Loading SFK table file {}'.format(file_ref))


df_pred = df_pred.drop(columns= ['prequel_count', 'prequel_featured_count']).copy()
df_pred = pd.merge(df_pred, df_imdb[['imdb_series_id', 'prequel_count', 'prequel_featured_count']], 
                   on=['imdb_series_id'], how='left').copy()
df_pred[['prequel_count', 'prequel_featured_count']] = df_pred[['prequel_count', 'prequel_featured_count']].fillna(0)

## 1.1 Train Test Scope

In [0]:
# Train_Test_Scope = 'lib/dev_Post_GreenLight_Model.py'
# %run $Train_Test_Scope

In [0]:
from lib.dev_train_test_scope import train_test_scope

current_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
train_test_data = train_test_scope(current_date, input_bucket, output_bucket, 
                                   database, schema, geo_value, kpi, metadata_feature, df_pred)
train_test_data.run()
train_dataset = train_test_data.train_dataset
score_pgl = train_test_data.score_pgl
score_pre = train_test_data.score_pre
score_post = train_test_data.score_post

## Model

## 2.0 Post-GreenLight Model

In [0]:
from lib.dev_Post_GreenLight_Model import post_greenlight_model

### 2.1 Pull in Data

In [0]:
# Name train and set
train_data_set = train_test_data.train_dataset
test_data_set = train_test_data.score_pgl

# Import Engagement
query_metric = f"""select * from {database}.content_datascience.viewingsubs_metrics_train where geo_value='{geo_value}'"""
logger.info(f'{query_metric}')
df_metric = sfk.execute_query(query = query_metric, ctx = ctx)

In [0]:
df_metric.info()

### 2.2 Run Model

In [0]:
Post_GreenLight_Model = 'lib/dev_Post_GreenLight_Model.py'
%run $Post_GreenLight_Model

In [0]:
# Run Model with Cross Validation - 
print('POST WAVE II EMEA ONLY')
pgl_model = post_greenlight_model(train_data_set, test_data_set, kpi, 
                                  input_bucket, output_bucket, geo_value, 
                                  database, schema, df_metric)
pgl_model.feature_engineer()
input_train = pgl_model.train_data
input_test = pgl_model.test_data
pgl_model.cv(NUM_FOLD = 5)

# To review predictions and feature importances in notebook
feature_importances = pgl_model.feature_importances
validation_set = pgl_model.validation_set

In [0]:
# Run Model with Cross Validation
pgl_model = post_greenlight_model(train_data_set, test_data_set, kpi, 
                                  input_bucket, output_bucket, geo_value, 
                                  database, schema, df_metric)
pgl_model.feature_engineer()
input_train = pgl_model.train_data
input_test = pgl_model.test_data
pgl_model.cv(NUM_FOLD = 5)

# To review predictions and feature importances in notebook
feature_importances = pgl_model.feature_importances
validation_set = pgl_model.validation_set

In [0]:
pgl_model.scoring()
prediction_set_post_gl = pgl_model.prediction_set

In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [0]:
from sklearn.metrics import r2_score

for fold in validation_set['fold'].drop_duplicates().to_list():
    sample = len(validation_set.loc[(validation_set['fold']==fold), 'viewing_subs'].to_list())
    r2 = r2_score(validation_set.loc[(validation_set['fold']==fold), 'viewing_subs'].to_list(),
                    validation_set.loc[(validation_set['fold']==fold), 'prediction'].to_list())

    mape = mean_absolute_percentage_error(validation_set.loc[(validation_set['fold']==fold), 'viewing_subs'],
                                         validation_set.loc[(validation_set['fold']==fold), 'prediction'])
    logger.info(f'Fold {fold}: Train Size {sample} | R2 {r2.round(2)} | MAPE {mape.round()}')

In [0]:
from sklearn.metrics import r2_score
logger.info(f'POST WAVE 2 2022-03-08 training data only')
for fold in validation_set['fold'].drop_duplicates().to_list():
    sample = len(validation_set.loc[(validation_set['fold']==fold), 'viewing_subs'].to_list())
    r2 = r2_score(validation_set.loc[(validation_set['fold']==fold), 'viewing_subs'].to_list(),
                    validation_set.loc[(validation_set['fold']==fold), 'prediction'].to_list())

    mape = mean_absolute_percentage_error(validation_set.loc[(validation_set['fold']==fold), 'viewing_subs'],
                                         validation_set.loc[(validation_set['fold']==fold), 'prediction'])
    logger.info(f'Fold {fold}: Train Size {sample} | R2 {r2.round(2)} | MAPE {mape.round()}')

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
# Compare Old Dataset to new dataset
# fig = plt.figure(figsize=(7,3))
x = f'viewing_subs'
y = f'prediction'
data = validation_set
sns.set_theme(style="whitegrid")
ax = sns.scatterplot(data=data, x=x, y=y, 
                     hue = 'observed_medal_num',
                     palette = 'muted'
#                      fold = 'tier',
#                      row='observed_medal_num'
               )
# plt.title(f'Viewing Subs - 28 Training Data')
plt.axvline(x=0, linewidth=.5, color='black', ls= '--')
plt.axhline(y=0, linewidth=.5, color='black', ls= '--')

plt.xlim(-1,60)
plt.ylim(-1,60)
sns.move_legend(
    ax, "lower center",
    bbox_to_anchor=(.5, 1), ncol=4, title='Tiers', frameon=False,
)
plt.tight_layout()

## 3.0 Pre-Launch Model

### Import Data

In [0]:
dev_pre_launch_model = 'lib/dev_Pre_Launch_Model.py'
%run $dev_pre_launch_model

In [0]:
# Import Datasets

# Import Engagement
# query_metric = f"""select * from {database}.content_datascience.viewingsubs_metrics_train where geo_value='{geo_value}'"""
# logger.info(f'{query_metric}')
# df_metric = sfk.execute_query(query = query_metric, ctx = ctx)

# Import Wikipedia Data
query_wiki_train = f"""select * 
from {database}.content_datascience.title_season_wiki_daily_region 
where geo_value='{geo_value}'
and days_from_premiere >= -28 and days_from_premiere <=0"""
logger.info(f'{query_metric}')
df_wiki_train = sfk.execute_query(query = query_wiki_train, ctx = ctx)

# Import Future Wiki Data
query_wiki_pred = f"""select * 
from {database}.content_datascience.title_season_wiki_daily_pred_test
where geo_value='{geo_value}'
and days_from_premiere >= -28 and days_from_premiere <=0
"""
# logger.info(f'{wiki_query_pred}')
df_wiki_pred = sfk.execute_query(query = query_wiki_pred, ctx = ctx)

In [0]:
df_wiki_pred

In [0]:
df_wiki_train

In [0]:
# Name train and set
train_dataset = train_test_data.train_dataset
test_dataset = train_test_data.score_pgl

# Combine train and prediction wikipedia data
wiki_columns = ['title_series','title_season','season_number', 'imdb_series_id',
                'first_release_date', 'days_from_premiere', 'request_date', 'page_views']
df_wiki_train['first_release_date'] = pd.to_datetime(df_wiki_train['first_release_date']).dt.strftime('%Y-%m-%d')
df_wiki_train = df_wiki_train[wiki_columns].drop_duplicates()
df_wiki_pred = df_wiki_pred[wiki_columns].drop_duplicates()

df_wiki_train['type'] = 'train'
df_wiki_pred['type'] = 'test'
df_wiki = pd.concat([df_wiki_train, df_wiki_pred]).drop_duplicates().reset_index(drop=True)


## Run Model

In [0]:
dev_pre_launch_model = 'lib/dev_Pre_Launch_Model.py'
%run $dev_pre_launch_model

In [0]:
test_dataset

In [0]:
pre_model = pre_launch_model(train_dataset, test_dataset, kpi, current_date, input_bucket, output_bucket,
                            geo_value, database, df_metric, df_wiki)
pre_model.feature_engineer()
pre_model.cv(NUM_FOLD = 1)
# pre_model.scoring()

In [0]:
pre_model.cv(NUM_FOLD = 2)

In [0]:
validation_set = Utils.read_csv_s3(bucket=output_bucket, key_path=kpi, filename=f'validation_set_prelaunch_new_{geo_value}.csv').drop(columns=['Unnamed: 0'])

In [0]:
from sklearn.metrics import r2_score

for day in np.arange(-27, 0):
    r2 = r2_score(validation_set.loc[(validation_set['days_from_premiere']==day), 'viewing_subs'].to_list(),
                    validation_set.loc[(validation_set['days_from_premiere']==day), 'prediction'].to_list())
    print(f'R2 for {day}: {r2.round(2)}')

### 2.3 Format PostGLight Predictions and Write to Delphi

In [0]:
# Add Columns needed for Delphi Prediction Tables
# df_delphi_postgl = df_delphi.merge(prediction_set_post_gl, on='key').drop('key', 1).reset_index(drop=True).copy()

data = {
    'model_name': ['pct_viewing_subs_2.0'],
    'table_name': ['pct_viewing_subs_postgl'],
    'model_version': ['2.0'],
    'sub_type': ['Max Retail+Wholesale'],
    'sub_plan': ['Platform'],
    'unit': ['percent'],
    'region': [geo_value],
    'days_after_premiere': [28],
    'publish_date': [current_date],
    'key': [1]
}
df_delphi = pd.DataFrame(data=data, columns = data.keys())


prediction_set_post_gl['key'] = 1
df_delphi_postgl = df_delphi.merge(prediction_set_post_gl, on='key').drop(
    'key', 1).reset_index(drop=True).copy()

# Add imdb_id and ckg_match_id
df_delphi_postgl = df_delphi_postgl.merge(
    test_data_set[['delphi_id', 'ckg_match_id', 'ckg_series_id', 'imdb_series_id',
                   'title_season', 'title_series', 'season_number']], 
    on=['title_season', 'title_series', 'season_number'], how='inner')

In [0]:
# Rename Columns to match Delphi
rename_set = {
    'ckg_series_id' : 'title_id',
    'imdb_series_id' : 'imdb_id',
    'title_series' : 'title_name', 
    'derived_genre' :'category',   
    'first_release_date' : 'premiere_date'
}
df_delphi_postgl.rename(columns = rename_set, inplace=True)


In [0]:
# Update dates to datetime object
df_delphi_postgl['premiere_date'] = pd.to_datetime(df_delphi_postgl['premiere_date'])
df_delphi_postgl['publish_date'] = pd.to_datetime(df_delphi_postgl['publish_date'])
df_delphi_postgl['current_days_from_premiere'] = (df_delphi_postgl['publish_date'] - df_delphi_postgl['premiere_date']).dt.days
df_delphi_postgl['target_date'] = df_delphi_postgl['premiere_date'] + df_delphi_postgl['days_after_premiere'].apply(lambda x: pd.DateOffset(days=x))

# Back to string to make compatible with sfk package
df_delphi_postgl['publish_date'] = df_delphi_postgl['publish_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_delphi_postgl['target_date'] = df_delphi_postgl['target_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_delphi_postgl['premiere_date'] = df_delphi_postgl['premiere_date'].apply(lambda x: x.strftime('%Y-%m-%d'))


In [0]:
# Write to Snowflake
df_delphi_postgl = df_delphi_postgl[df_delphi_postgl['delphi_id'].notnull()].reset_index(drop=True).copy()
# sfk.export_dataframe_to_table(database=database, schema='delphi', df=df_delphi_postgl, 
#                               table=f'{df_delphi_postgl.loc[0,"table_name"]}_staging', conn=ctx)

In [0]:
from lib.dev_Pre_Launch_Model import post_greenlight_model

In [0]:
dev_pre_launch_model = 'lib/dev_Pre_Launch_Model.py'
%run $dev_pre_launch_model


In [0]:
pre_launch_model = 'lib/Pre_Launch_Model.py'
%run $pre_launch_model

### 3.1 Pull in Data

In [0]:
# create or replace table max_dev.workspace.forecasting_signals_search as (
# select gs.TRACKING_GROUP_ID as imdb_id, date,indexed_volume::float as value, 'search' as metric,geo_name
# ,CONVERT_TIMEZONE('UTC', 'America/Los_Angeles',current_timestamp::TIMESTAMP_NTZ)::timestamp  as create_ts
# from "MAX_PROD"."CKG"."GST_COUNTRY_LEVEL_VOLUME" as gs
# where date::date > '2020-01-01' 
# and gs.TRACKING_GROUP_ID in (select imdb_id from forecasting_premieres)
# and lower(geo_name) in (select lower(country_iso_code) from forecasting_geo_map)
# );